In [ ]:
import logging

# Our imports
import emission.core.get_database as edb
import emission.analysis.modelling.tour_model.cluster_pipeline as pipeline
import emission.analysis.modelling.tour_model.similarity as similarity
import emission.analysis.modelling.tour_model.featurization as featurization
import emission.analysis.modelling.tour_model.representatives as representatives
import emission.storage.decorations.analysis_timeseries_queries as esda
import pandas as pd
from numpy import *
import confirmed_trips_eval_bins_clusters as evaluation
from sklearn import metrics
from pandas.testing import assert_frame_equal

In [ ]:
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)

In [ ]:
participant_uuid_obj = list(edb.get_profile_db().find({"install_group": "participant"}, {"user_id": 1, "_id": 0}))
all_users = [u["user_id"] for u in participant_uuid_obj]

In [ ]:
radius = 100

In [ ]:
query_day = []

In [ ]:
query_month = []

In [ ]:
def match_day(trip,bin):
    if bin:
        t = filter_trips[bin[0]]
        if trip['data']['start_local_dt']['year']==t['data']['start_local_dt']['year']and trip['data']['start_local_dt']['month']==t['data']['start_local_dt']['month']and trip['data']['start_local_dt']['day']==t['data']['start_local_dt']['day']:
            return True
    return False

In [ ]:
def match_month(trip,bin):
    if bin:
        t = filter_trips[bin[0]]
        if trip['data']['start_local_dt']['year']==t['data']['start_local_dt']['year']and trip['data']['start_local_dt']['month']==t['data']['start_local_dt']['month']:
            return True
    return False

In [ ]:
for i in range(len(all_users)):
    user = all_users[i]
    filter_trips,sim = evaluation.filter_data(user,radius)
    logging.debug("len(filter_trips)is %s "% len(filter_trips))

    # filter out users that don't have valid labeled trips
    if len(filter_trips) == 0:
        query_day.append(NaN)
        query_month.append(NaN)
        continue
        
    sim.bin_data()
    sim.delete_bins()
    # bins below cutoff
    bl_bins = sim.below_cutoff
    
    # a list of trip indices from bl_bins
    bl_trip_ls = []
    for bin in bl_bins:
        for index in bin:
            bl_trip_ls.append(index)

    # collect all the trips that below cutoff
    bl_trips = [filter_trips[num]for num in bl_trip_ls]
    
    
    # collect query times in a day
    bin_day = []
    for trip_index in bl_trip_ls:
        added = False
        trip = filter_trips[trip_index]
        for bin in bin_day:
            if match_day(trip,bin):
                bin.append(trip_index)
                added = True
        if not added:
            bin_day.append([trip_index])
    query_day_ls = []
    for bin in bin_day:
        query_day_ls.append(len(bin))

    # average query times for a day
    query_day.append(math.ceil(mean(query_day_ls)))
    
    
    # collect query times in a month
    bin_month = []
    for trip_index in bl_trip_ls:
        added = False
        trip = filter_trips[trip_index]
        for bin in bin_month:
            if match_month(trip,bin):
                bin.append(trip_index)
                added = True
        if not added:
            bin_month.append([trip_index])
        query_month_ls = []
    for bin in bin_month:
        query_month_ls.append(len(bin))

    # average query times for a month
    query_month.append(math.ceil(mean(query_month_ls)))


In [ ]:
query_day

In [ ]:
query_month

In [ ]:
mean_day = math.ceil(mean([x for x in query_day if str(x) != 'nan']))
mean_day

In [ ]:
mean_month = math.ceil(mean([x for x in query_month if str(x) != 'nan']))
mean_month

In [ ]:
df = pd.DataFrame(data = {'query times in a day':query_day,'query times in a month':query_month})
df.loc['mean'] = [mean_day,mean_month]
df

In [ ]:
df.plot(kind = 'bar')